# Preprocessing

In [1]:
!pip install transformers datasets faiss-cpu sentence-transformers
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
import faiss
import numpy as np


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 56.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled n

2025-06-11 15:04:57.704559: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749654297.905808      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749654297.959252      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [50]:
# # Load all four datasets
# ds1 = load_dataset("KisanVaani/agriculture-qa-english-only")
# ds2 = load_dataset("Mahesh2841/Agriculture")
# ds3 = load_dataset("YuvrajSingh9886/Agriculture-Soil-QA-Pairs-Dataset")
# ds4 = load_dataset("muhammad-atif-ali/agriculture-qa-english-llama-2")
# ds5 = load_dataset("kshubham/agriculture_data_5k")

#Example dataset (replace with actual)
dataset = load_dataset("KisanVaani/agriculture-qa-english-only", split="train")

# Extract questions and answers
questions = dataset["question"]
answers = dataset["answers"]

# Load sentence transformer for embeddings
retriever_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Create embeddings
question_embeddings = retriever_model.encode(questions, show_progress_bar=True)


Batches:   0%|          | 0/707 [00:00<?, ?it/s]

In [33]:
!pip install transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model and tokenizer
model_name = "google/flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)




In [45]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load model/tokenizer
model_name = "google/flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

def generate_context_t5(question, answer, max_tokens=128):
    prompt = (
        f"Write a paragraph that answers the question: '{question}' "
        f"and includes this exact phrase: '{answer}'."
    )

    # Tokenize, then move to device correctly
    encoded = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=256)
    encoded = {k: v.to(device) for k, v in encoded.items()}

    # Generate
    output = model.generate(**encoded, max_length=max_tokens)

    # Decode
    paragraph = tokenizer.decode(output[0], skip_special_tokens=True)

    # Check if answer is in paragraph
    if answer in paragraph:
        return paragraph.strip()
    else:
        return None


Using device: cuda


In [46]:
print(model.device)
print(inputs["input_ids"].device)


cuda:0


NameError: name 'inputs' is not defined

In [49]:
from datasets import Dataset
from tqdm.notebook import tqdm
generated_contexts = []
for q, a in tqdm(zip(questions, answers), total=len(questions), desc="Generating contexts"):
    context = generate_context_t5(q, a)
    if context:
        generated_contexts.append({
            "question": q,
            "context": context,
            "answers": {
                "text": [a],
                "answer_start": [context.find(a)]
            }
        })

dataset = Dataset.from_list(generated_contexts)

def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    contexts = [c.strip() for c in examples["context"]]
    answers = examples["answers"]

    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
        stride=128,
        return_overflowing_tokens=True
    )

    start_positions = []
    end_positions = []

    for i, offset in enumerate(inputs["offset_mapping"]):
        input_ids = inputs["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = inputs.sequence_ids(i)

        # Handle batch/overflow mapping
        sample_idx = inputs["overflow_to_sample_mapping"][i]
        answer = answers[sample_idx]["text"][0]
        start_char = answers[sample_idx]["answer_start"][0]
        end_char = start_char + len(answer)

        if sequence_ids is None or 1 not in sequence_ids:
            start_positions.append(cls_index)
            end_positions.append(cls_index)
            continue

        try:
            token_start_index = sequence_ids.index(1)
            token_end_index = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)
        except ValueError:
            start_positions.append(cls_index)
            end_positions.append(cls_index)
            continue

        if not (offset[token_start_index][0] <= start_char and offset[token_end_index][1] >= end_char):
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            for idx in range(token_start_index, token_end_index + 1):
                if offset[idx][0] <= start_char < offset[idx][1]:
                    start_positions.append(idx)
                    break
            else:
                start_positions.append(cls_index)

            for idx in range(token_end_index, token_start_index - 1, -1):
                if offset[idx][0] < end_char <= offset[idx][1]:
                    end_positions.append(idx)
                    break
            else:
                end_positions.append(cls_index)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    inputs.pop("offset_mapping")

    return inputs



Generating contexts:   0%|          | 0/22615 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
tokenized_dataset = dataset.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=dataset.column_names
)


In [4]:
def normalize_dataset(dataset, split="train"):
    texts = []
    dataset_name = dataset[split].name if hasattr(dataset[split], 'name') else "Unknown"

    # Dataset 1: KisanVaani/agriculture-qa-english-only (question, answers)
    if "question" in dataset[split].features and "answers" in dataset[split].features:
        for item in dataset[split]:
            question = item["question"]
            # Handle if answers is a list or string
            answer = item["answers"] if isinstance(item["answers"], str) else " ".join(item["answers"]) if item["answers"] else "No answer provided."
            text = f"Question: {question}\nAnswer: {answer}"
            texts.append(text)

    # Dataset 2 & 5: Mahesh2841/Agriculture, kshubham/agriculture_data_5k (instruction, input, response)
    elif "instruction" in dataset[split].features and "response" in dataset[split].features:
        for item in dataset[split]:
            instruction = item["instruction"] or ""
            input_text = item["input"] or ""
            response = item["response"] or "No response provided."
            # Combine into a single text
            text = f"Instruction: {instruction}\nInput: {input_text}\nResponse: {response}"
            texts.append(text)

    # Dataset 3: YuvrajSingh9886/Agriculture-Soil-QA-Pairs-Dataset (QUESTION.question, ANSWER, QUESTION.paragraph)
    elif "QUESTION.question" in dataset[split].features and "ANSWER" in dataset[split].features:
        for item in dataset[split]:
            question = item["QUESTION.question"] or ""
            answer = item["ANSWER"] or "No answer provided."
            paragraph = item["QUESTION.paragraph"] or ""
            # Include paragraph as context if available
            text = f"Context: {paragraph}\nQuestion: {question}\nAnswer: {answer}" if paragraph else f"Question: {question}\nAnswer: {answer}"
            texts.append(text)

    # Dataset 4: muhammad-atif-ali/agriculture-qa-english-llama-2 (formatted_text)
    elif "formatted_text" in dataset[split].features:
        for item in dataset[split]:
            formatted_text = item["formatted_text"] or ""
            # Check if it's already in Q&A format, otherwise treat as raw text
            if "Question:" in formatted_text and "Answer:" in formatted_text:
                texts.append(formatted_text)
            else:
                texts.append(f"Text: {formatted_text}")

    else:
        print(f"Unknown format for dataset {dataset_name}: {dataset[split].features}")
        return []

    print(f"Extracted {len(texts)} documents from {dataset_name}")
    return texts

# Normalize all datasets
texts1 = normalize_dataset(ds1)
texts2 = normalize_dataset(ds2)
texts3 = normalize_dataset(ds3)
texts4 = normalize_dataset(ds4)
texts5 = normalize_dataset(ds5)

# Combine all texts into a single list
all_texts = texts1 + texts2 + texts3 + texts4 + texts5
print(f"Total documents combined: {len(all_texts)}")

Extracted 22615 documents from Unknown
Extracted 5916 documents from Unknown
Extracted 3447 documents from Unknown
Extracted 22615 documents from Unknown
Extracted 5916 documents from Unknown
Total documents combined: 60509


# For Fine-Tuning

In [51]:
from datasets import Dataset

qa_data = []

for q, a in zip(questions, answers):
    if a.strip() and q.strip():
        qa_data.append({
            "question": q.strip(),
            "context": a.strip(),     # using answer as context
            "answers": {
                "text": [a.strip()],
                "answer_start": [0]   # always 0 since answer == context
            }
        })

dataset = Dataset.from_list(qa_data)


In [52]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering


model_checkpoint = "deepset/bert-base-cased-squad2"  # or squad2 tuned: deepset/bert-base-cased-squad2

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)


Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [55]:
from transformers import DefaultDataCollator

def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    contexts = [c.strip() for c in examples["context"]]
    answers = examples["answers"]

    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
        stride=128,
        return_overflowing_tokens=True
    )

    start_positions = []
    end_positions = []

    for i, offset in enumerate(inputs["offset_mapping"]):
        input_ids = inputs["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = inputs.sequence_ids(i)

        # Handle batch/overflow mapping
        sample_idx = inputs["overflow_to_sample_mapping"][i]
        answer = answers[sample_idx]["text"][0]
        start_char = answers[sample_idx]["answer_start"][0]
        end_char = start_char + len(answer)

        if sequence_ids is None or 1 not in sequence_ids:
            start_positions.append(cls_index)
            end_positions.append(cls_index)
            continue

        try:
            token_start_index = sequence_ids.index(1)
            token_end_index = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)
        except ValueError:
            start_positions.append(cls_index)
            end_positions.append(cls_index)
            continue

        if not (offset[token_start_index][0] <= start_char and offset[token_end_index][1] >= end_char):
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            for idx in range(token_start_index, token_end_index + 1):
                if offset[idx][0] <= start_char < offset[idx][1]:
                    start_positions.append(idx)
                    break
            else:
                start_positions.append(cls_index)

            for idx in range(token_end_index, token_start_index - 1, -1):
                if offset[idx][0] < end_char <= offset[idx][1]:
                    end_positions.append(idx)
                    break
            else:
                end_positions.append(cls_index)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    inputs.pop("offset_mapping")

    return inputs



tokenized_dataset = dataset.map(preprocess_training_examples, batched=True, remove_columns=dataset.column_names)
data_collator = DefaultDataCollator()


Map:   0%|          | 0/22615 [00:00<?, ? examples/s]

In [56]:
import torch
print("Dataset size:", len(tokenized_dataset))

print("Using device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


Dataset size: 22639
Using device: Tesla T4


In [57]:
from transformers import TrainingArguments, Trainer
import torch
# Step 1: Verify GPU availability (T4 x2)
print(f"GPU Available: {torch.cuda.is_available()}")
print(f"Number of GPUs: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
if torch.cuda.is_available():
    model = model.cuda()
    
training_args = TrainingArguments(
    output_dir="./agri-qa-model",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    logging_strategy="steps",
    logging_steps=500,  # log every step
    save_steps=1000,
    disable_tqdm=False,
    report_to="none",  # avoid wandb overhead
    fp16=torch.cuda.is_available()
)


print("hi")
# Calculate total training steps for progress bar

# Custom callback for progress bar
from transformers import TrainerCallback

class PrinterCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        print(f">>> {logs}", flush=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[PrinterCallback()]
)

print("Tokenized dataset size:", len(tokenized_dataset))
if len(tokenized_dataset) == 0:
    raise ValueError("Dataset is empty — training will not run.")

print("Starting..")
trainer.train()
print("Finished")

GPU Available: True
Number of GPUs: 2
GPU 0: Tesla T4
GPU 1: Tesla T4
hi
Tokenized dataset size: 22639
Starting..


/tmp/ipykernel_35/1853878575.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
500,0.043300
1000,0.002200


>>> {'loss': 0.0433, 'grad_norm': 0.06463667005300522, 'learning_rate': 3.236749116607774e-05, 'epoch': 0.35335689045936397}
>>> {'loss': 0.0022, 'grad_norm': 0.05030721798539162, 'learning_rate': 1.4699646643109541e-05, 'epoch': 0.7067137809187279}


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


>>> {'train_runtime': 973.4234, 'train_samples_per_second': 23.257, 'train_steps_per_second': 1.454, 'total_flos': 4436622956809728.0, 'train_loss': 0.016061367083985898, 'epoch': 1.0}
Finished


In [12]:
trainer.save_model("./final_agri_qa_model")
tokenizer.save_pretrained("./final_agri_qa_model")

('./final_agri_qa_model/tokenizer_config.json',
 './final_agri_qa_model/special_tokens_map.json',
 './final_agri_qa_model/vocab.txt',
 './final_agri_qa_model/added_tokens.json',
 './final_agri_qa_model/tokenizer.json')

# For RAG

In [2]:
dataset = load_dataset("KisanVaani/agriculture-qa-english-only", split="train")

# Extract questions and answers
questions = dataset["question"]
answers = dataset["answers"]

README.md:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.97M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/22615 [00:00<?, ? examples/s]

In [26]:
questions = []
answers = []


# s = "qn: ansfjf\nans: sdvjsdv"
# q_line = s.split("\n")[0].replace("qn: ", "").strip()
# a_line = s.split("\n")[1].replace("ans: ", "").strip()
print(q_line)
print(a_line)
for item in :
    new = item.split("\n")
    if len(new)>1:
        q_line = new[0].replace("Question: ", "").strip()
        a_line = new[1].replace("Answer: ", "").strip()
        questions.append(q_line)
        answers.append(a_line)

Instruction: You are an experienced agronomist proficient in farming techniques, crop management, and disease-resistant crop cultivation, you are tasked with answering questions based on your expertise.
Input: Attracting beneficial wildlife to a monoculture crop field.


In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("recobo/agriculture-bert-uncased")
model = AutoModelForMaskedLM.from_pretrained("recobo/agriculture-bert-uncased")

In [5]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
# Load sentence transformer for embeddings
retriever_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Create embeddings
question_embeddings = retriever_model.encode(questions, show_progress_bar=True)
dimension = question_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(question_embeddings))
# Load a pretrained QA model
qa_pipeline = pipeline("question-answering", 
                       model=AutoModelForMaskedLM.from_pretrained("recobo/agriculture-bert-uncased"),
                       tokenizer=AutoTokenizer.from_pretrained("recobo/agriculture-bert-uncased"))




modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

The model 'BertForMaskedLM' is not supported for question-answering. Supported models are ['AlbertForQuestionAnswering', 'BartForQuestionAnswering', 'BertForQuestionAnswering', 'BigBirdForQuestionAnswering', 'BigBirdPegasusForQuestionAnswering', 'BloomForQuestionAnswering', 'CamembertForQuestionAnswering', 'CanineForQuestionAnswering', 'ConvBertForQuestionAnswering', 'Data2VecTextForQuestionAnswering', 'DebertaForQuestionAnswering', 'DebertaV2ForQuestionAnswering', 'DiffLlamaForQuestionAnswering', 'DistilBertForQuestionAnswering', 'ElectraForQuestionAnswering', 'ErnieForQuestionAnswering', 'ErnieMForQuestionAnswering', 'FalconForQuestionAnswering', 'FlaubertForQuestionAnsweringSimple', 'FNetForQuestionAnswering', 'FunnelForQuestionAnswering', 'GPT2ForQuestionAnswering', 'GPTNeoForQuestionAnswering', 'GPTNeoXForQuestionAnswering', 'GPTJForQuestionAnswering', 'IBertForQuestionAnswering', 'LayoutLMv2ForQuestionAnswering', 'LayoutLMv3ForQuestionAnswering', 'LEDForQuestionAnswering', 'LiltF

Ask an agri question (or 'exit'):  what is crop rotation?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:390: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


KeyError: 'start_logits'

# Testing

In [6]:
def ask_agri_bot(user_question, top_k=3):
    # Encode user query
    query_embedding = retriever_model.encode([user_question])
    
    # Search similar questions
    _, similar_indices = index.search(np.array(query_embedding), top_k)
    
    # Use similar answers as context
    context = " ".join([answers[idx] for idx in similar_indices[0]])
    
    # Generate answer using QA model
    result = qa_pipeline({
        'question': user_question,
        'context': context
    })
    
    return result['answer']


while True:
    q = input("Ask an agri question (or 'exit'): ")
    if q.lower() == 'exit':
        break
    print("Answer:", ask_agri_bot(q))


Ask an agri question (or 'exit'):  crop rotation


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

KeyError: 'start_logits'

In [16]:
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
tokenizer = AutoTokenizer.from_pretrained("./final_agri_qa_model")
model = AutoModelForQuestionAnswering.from_pretrained("./final_agri_qa_model")
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [58]:
import os

# Find all checkpoints in the results directory
checkpoints = [f for f in os.listdir("/kaggle/working/agri-qa-model") if f.startswith("checkpoint-")]
checkpoints = sorted(checkpoints, key=lambda x: int(x.split("-")[-1]))  # sort by step number

latest_checkpoint = os.path.join("/kaggle/working/agri-qa-model/checkpoint-1415")
print("Using checkpoint:", latest_checkpoint)

from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained(latest_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(latest_checkpoint)
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)


Device set to use cuda:0


Using checkpoint: /kaggle/working/agri-qa-model/checkpoint-1415


In [ ]:
while True:
    q = input("Ask your question (or type 'exit'): ")
    if q.lower() == 'exit':
        break
    result = qa_pipeline({"question": q, "context": context})
    print("Answer:", result["answer"])


Ask your question (or type 'exit'):  what is crop rotation?


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:390: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


Answer: .


Ask your question (or type 'exit'):  What farming practice helps prevent soil erosion?


Answer: .


Ask your question (or type 'exit'):  exit


# Saving files

In [9]:
retriever_model.save("retriever_model")
faiss.write_index(index, "faiss_index.index")
import pickle

with open("retrieval_answers.pkl", "wb") as f:
    pickle.dump(answers, f)
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

model_name = "deepset/bert-base-cased-squad2"
qa_model = AutoModelForQuestionAnswering.from_pretrained(model_name)
qa_tokenizer = AutoTokenizer.from_pretrained(model_name)

qa_model.save_pretrained("qa_model")
qa_tokenizer.save_pretrained("qa_model")


Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


('qa_model/tokenizer_config.json',
 'qa_model/special_tokens_map.json',
 'qa_model/vocab.txt',
 'qa_model/added_tokens.json',
 'qa_model/tokenizer.json')

In [12]:
import shutil

shutil.make_archive("retriever_model", 'zip', root_dir='/kaggle/working/retriever_model')


'/kaggle/working/retriever_model.zip'